# Visualization Strawman

(18-May-2022 jat) This is an exemplar for some of the features to be prototyped in Phase 1. The main goal of our software should be to make it **easy** for notebook authors to render 3-D data arrays.

## 1. Load test data

* Set `dataset_path` to the path on your file system
* Future: make datasets like this available on kwnas and/or internal http server?

In [ ]:
# Load test dataset
import xarray as xr
import zarr

dataset_path = '/home/john/projects/pan3d/data/test'
ds = xr.open_zarr(dataset_path, consolidated=False)

## 2. Select data array

In [16]:
temp_data = ds.T
temp_data

<xarray.DataArray 'T' (time: 396, Z: 40, XC_agg: 20, YC_agg: 20)>
[6336000 values with dtype=float32]
Coordinates:
    PHrefC   (Z) float32 49.05 147.1 245.2 ... 2.676e+04 2.777e+04 2.878e+04
  * XC_agg   (XC_agg) float32 5e+04 1.5e+05 2.5e+05 ... 1.85e+06 1.95e+06
  * YC_agg   (YC_agg) float32 5e+04 1.5e+05 2.5e+05 ... 1.85e+06 1.95e+06
  * Z        (Z) float32 -5.0 -15.0 -25.0 ... -2.728e+03 -2.83e+03 -2.934e+03
    drF      (Z) float32 10.0 10.0 10.0 12.0 14.0 ... 103.0 103.0 103.0 103.0
    dxCc     (YC_agg, XC_agg) float32 1e+05 1e+05 1e+05 ... 1e+05 1e+05 1e+05
    dyCc     (YC_agg, XC_agg) float32 1e+05 1e+05 1e+05 ... 1e+05 1e+05 1e+05
    hFacC    (Z, XC_agg, YC_agg) float32 0.95 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
  * time     (time) timedelta64[ns] 61023 days 61038 days ... 66948 days

## 3. Create vtkDataObject instance from data array at one timestep
* Presumes there is a hypothetical `pan3d` module that encapsulates vtk and pyvista as needed.
* So no, this code wont run.
* `pan3d.to_vtk()` recognizes that the data array represents a rectilinear grid so, in this case, it returns a `vtkRectiliearGrid` object.
* Technically, the code returns a python subclass of `vtkRectilinearGrid` that uses an xarray accessor (to be developed) so that no data copying is required.

In [ ]:
vtk_grid = pan3d.to_vtk(temp_data[0]) # returns vtkRectilinearGrid (subclass)

## 4. Plot data array at one timestep
* In this cell, `pan3d.plot()` internally creates the `vtkRectilinearGrid` instance and rendering pipeline.
* `pan3d.plot()` options are TBD but should at least include color map, opacity map, client or server rendering mode, ...

In [ ]:
import pan3d
i = len(ds.T.time) // 2  # use middle time index
plot1 = pan3d.plot(temp_data[i])
plot1

## 5. Plot data array with timestep control
* Because the data array has time coordinates, `pan3d.plot()` displays a basic time slider by default.
* The code also provides an option to set its time slider to an external ipywidget (standard slider, custom vcr-style widget, whatever)
* The test data does not seem to be tied to a particular calendar date, so it's ok to display raw indices (0-395)
* Future: pan3d provides a calendar-aware slider that includes labeled tick marks on the axis (days, months, years, depending on range). Should also have popup calendar picker on double-click.

In [ ]:
plot2 = pan3d.plot(temp_data)

import ipywidgets as widgets
plot2.time_slider = widget.IntSlider(...) # for example
plot2